# LHM23 File Optimization Notebook

**Purpose:** Optimize the LHM23 file to serve as the singular source of truth for LHM & Bob business operations.

**Goals:**
- Eliminate redundancies
- Fix inaccuracies and contradictions
- Improve structure and organization
- Enhance maintainability and scalability
- Create a living document that grows with your business

---

## Step 1: Setup and File Loading

First, we'll set up the environment and load the LHM23 file for analysis.

In [4]:
import json
import os
from pathlib import Path
from datetime import datetime
from collections import defaultdict
import re

# Configuration
FILE_NAME = 'LHM23.json'  # Adjust based on your file format
BACKUP_DIR = 'backups'
OUTPUT_FILE = 'LHM23_optimized.json'

# Create backup directory if it doesn't exist
Path(BACKUP_DIR).mkdir(exist_ok=True)

print("Environment setup complete!")
print(f"Looking for file: {FILE_NAME}")
print(f"Backup directory: {BACKUP_DIR}")
print(f"Output file: {OUTPUT_FILE}")

Environment setup complete!
Looking for file: LHM23.json
Backup directory: backups
Output file: LHM23_optimized.json


In [5]:
# Function to load the LHM23 file (supports JSON, TXT, or other formats)
def load_lhm_file(filename):
    """
    Load the LHM23 file. Adjust based on your file format.
    """
    if not os.path.exists(filename):
        print(f"⚠️  File '{filename}' not found. Creating template structure...")
        return create_template_structure()
    
    file_ext = Path(filename).suffix.lower()
    
    try:
        if file_ext == '.json':
            with open(filename, 'r', encoding='utf-8') as f:
                return json.load(f)
        elif file_ext in ['.txt', '.md']:
            with open(filename, 'r', encoding='utf-8') as f:
                return {'raw_content': f.read()}
        else:
            with open(filename, 'r', encoding='utf-8') as f:
                return {'raw_content': f.read()}
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

# Try to load the file
lhm_data = load_lhm_file(FILE_NAME)

if lhm_data:
    print("✅ File loaded successfully!")
    print(f"Data type: {type(lhm_data)}")
    if isinstance(lhm_data, dict):
        print(f"Top-level keys: {list(lhm_data.keys())}")
else:
    print("❌ Failed to load file. Please check the filename and format.")

⚠️  File 'LHM23.json' not found. Creating template structure...


NameError: name 'create_template_structure' is not defined

## Step 2: Create Optimized Structure Template

Define the ideal structure for your LHM & Bob business source of truth.

In [ ]:
def create_template_structure():
    """
    Create an optimized template structure for LHM & Bob business data.
    This serves as the foundation for your single source of truth.
    """
    return {
        "metadata": {
            "version": "2.0",
            "last_updated": datetime.now().isoformat(),
            "created_by": "LHM23 Optimization Notebook",
            "description": "Single source of truth for LHM & Bob business operations"
        },
        
        "business_info": {
            "name": "LHM & Bob",
            "founded": "",
            "mission": "",
            "vision": "",
            "values": []
        },
        
        "products_services": {
            "offerings": [],
            "pricing": {},
            "descriptions": {}
        },
        
        "operations": {
            "processes": {},
            "workflows": {},
            "standard_procedures": {}
        },
        
        "customers": {
            "segments": [],
            "key_accounts": [],
            "contact_info": {}
        },
        
        "team": {
            "members": [],
            "roles": {},
            "responsibilities": {}
        },
        
        "financials": {
            "revenue_streams": [],
            "expenses": {},
            "goals": {}
        },
        
        "marketing": {
            "channels": [],
            "campaigns": [],
            "messaging": {}
        },
        
        "technology": {
            "tools": [],
            "systems": {},
            "integrations": []
        },
        
        "goals_metrics": {
            "short_term": [],
            "long_term": [],
            "kpis": {}
        },
        
        "notes": {
            "general": [],
            "ideas": [],
            "action_items": []
        }
    }

template = create_template_structure()
print("✅ Optimized structure template created!")
print(f"\nMain sections: {list(template.keys())}")

## Step 3: Analysis Functions

Create functions to analyze the current file for issues.

In [ ]:
def analyze_redundancies(data):
    """
    Identify duplicate or redundant information.
    """
    issues = []
    
    if isinstance(data, dict):
        # Check for duplicate values
        value_locations = defaultdict(list)
        
        def traverse(obj, path=""):
            if isinstance(obj, dict):
                for key, value in obj.items():
                    new_path = f"{path}.{key}" if path else key
                    traverse(value, new_path)
            elif isinstance(obj, list):
                for idx, item in enumerate(obj):
                    traverse(item, f"{path}[{idx}]")
            else:
                # Store primitive values and their locations
                if obj and str(obj).strip():
                    value_locations[str(obj)].append(path)
        
        traverse(data)
        
        # Find duplicates
        for value, locations in value_locations.items():
            if len(locations) > 1 and len(value) > 10:  # Only flag substantial duplicates
                issues.append({
                    "type": "redundancy",
                    "value": value[:50] + "..." if len(value) > 50 else value,
                    "locations": locations
                })
    
    return issues

def analyze_contradictions(data):
    """
    Look for potential contradictions (basic pattern matching).
    """
    issues = []
    
    # Keywords that might indicate contradictions
    contradiction_patterns = [
        (r'\bnot\b.*\bbut\b', 'Potential contradiction with "not...but"'),
        (r'\balthough\b.*\bhowever\b', 'Potential contradiction with "although...however"'),
        (r'\byes\b.*\bno\b', 'Conflicting yes/no statements')
    ]
    
    def search_text(obj, path=""):
        if isinstance(obj, str):
            for pattern, description in contradiction_patterns:
                if re.search(pattern, obj, re.IGNORECASE):
                    issues.append({
                        "type": "potential_contradiction",
                        "location": path,
                        "description": description,
                        "text": obj[:100]
                    })
        elif isinstance(obj, dict):
            for key, value in obj.items():
                search_text(value, f"{path}.{key}" if path else key)
        elif isinstance(obj, list):
            for idx, item in enumerate(obj):
                search_text(item, f"{path}[{idx}]")
    
    search_text(data)
    return issues

def analyze_completeness(data, template):
    """
    Check which sections are missing or incomplete.
    """
    missing = []
    
    def compare_structure(current, expected, path=""):
        if isinstance(expected, dict):
            for key in expected.keys():
                new_path = f"{path}.{key}" if path else key
                if key not in current:
                    missing.append({
                        "type": "missing_section",
                        "location": new_path
                    })
                else:
                    compare_structure(current.get(key, {}), expected[key], new_path)
    
    if isinstance(data, dict):
        compare_structure(data, template)
    
    return missing

print("✅ Analysis functions created!")

## Step 4: Run Analysis

Analyze the current LHM23 file for issues.

In [ ]:
# Run all analyses
print("🔍 Analyzing LHM23 file...\n")

if lhm_data:
    # Analyze redundancies
    redundancies = analyze_redundancies(lhm_data)
    print(f"📊 Redundancies found: {len(redundancies)}")
    if redundancies[:3]:  # Show first 3
        for r in redundancies[:3]:
            print(f"  - {r['value']} appears in {len(r['locations'])} locations")
    
    # Analyze contradictions
    contradictions = analyze_contradictions(lhm_data)
    print(f"\n⚠️  Potential contradictions: {len(contradictions)}")
    if contradictions[:3]:
        for c in contradictions[:3]:
            print(f"  - {c['description']} at {c['location']}")
    
    # Analyze completeness
    missing = analyze_completeness(lhm_data, template)
    print(f"\n📋 Missing sections: {len(missing)}")
    if missing[:5]:
        for m in missing[:5]:
            print(f"  - {m['location']}")
    
    # Summary
    print("\n" + "="*50)
    print("ANALYSIS SUMMARY")
    print("="*50)
    print(f"Total issues found: {len(redundancies) + len(contradictions) + len(missing)}")
    print(f"  - Redundancies: {len(redundancies)}")
    print(f"  - Contradictions: {len(contradictions)}")
    print(f"  - Missing sections: {len(missing)}")
else:
    print("⚠️  No data to analyze. Will create new optimized file from template.")

## Step 5: Merge and Optimize

Merge existing data with the optimized template structure.

In [ ]:
def merge_data(existing, template):
    """
    Intelligently merge existing data into the optimized template.
    Preserves all existing information while organizing it better.
    """
    result = template.copy()
    
    if not existing or not isinstance(existing, dict):
        return result
    
    def deep_merge(target, source):
        """Recursively merge source into target."""
        for key, value in source.items():
            if key in target:
                if isinstance(target[key], dict) and isinstance(value, dict):
                    deep_merge(target[key], value)
                elif isinstance(target[key], list) and isinstance(value, list):
                    # Merge lists, avoiding duplicates
                    for item in value:
                        if item not in target[key]:
                            target[key].append(item)
                else:
                    # Overwrite with existing data if it's not empty
                    if value:
                        target[key] = value
            else:
                # Add new keys from existing data
                target[key] = value
        return target
    
    result = deep_merge(result, existing)
    
    # Update metadata
    result["metadata"]["last_updated"] = datetime.now().isoformat()
    result["metadata"]["version"] = "2.0_optimized"
    
    return result

# Perform the merge
optimized_data = merge_data(lhm_data, template)

print("✅ Data merged and optimized!")
print(f"\nOptimized structure sections:")
for section in optimized_data.keys():
    print(f"  ✓ {section}")

## Step 6: Add Helper Functions

Create utility functions for maintaining the file.

In [ ]:
def add_entry(data, section, key, value):
    """
    Add a new entry to a specific section.
    """
    if section not in data:
        print(f"⚠️  Section '{section}' not found.")
        return data
    
    if isinstance(data[section], dict):
        data[section][key] = value
    elif isinstance(data[section], list):
        data[section].append(value)
    
    data["metadata"]["last_updated"] = datetime.now().isoformat()
    return data

def remove_duplicates(data):
    """
    Remove duplicate entries from lists.
    """
    def dedupe(obj):
        if isinstance(obj, dict):
            return {k: dedupe(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            # Remove duplicates while preserving order
            seen = set()
            result = []
            for item in obj:
                # Convert to string for comparison if not hashable
                try:
                    key = str(item) if not isinstance(item, (dict, list)) else json.dumps(item, sort_keys=True)
                except:
                    key = str(item)
                
                if key not in seen:
                    seen.add(key)
                    result.append(dedupe(item) if isinstance(item, (dict, list)) else item)
            return result
        else:
            return obj
    
    return dedupe(data)

def validate_data(data):
    """
    Validate the data structure.
    """
    issues = []
    
    # Check required sections
    required_sections = ["metadata", "business_info", "goals_metrics"]
    for section in required_sections:
        if section not in data:
            issues.append(f"Missing required section: {section}")
    
    # Check metadata
    if "metadata" in data:
        required_meta = ["version", "last_updated"]
        for field in required_meta:
            if field not in data["metadata"]:
                issues.append(f"Missing metadata field: {field}")
    
    return issues

print("✅ Helper functions created!")
print("\nAvailable functions:")
print("  - add_entry(data, section, key, value)")
print("  - remove_duplicates(data)")
print("  - validate_data(data)")

## Step 7: Final Optimization

Apply final optimizations and remove duplicates.

In [ ]:
# Remove duplicates
print("🔧 Removing duplicates...")
optimized_data = remove_duplicates(optimized_data)
print("✅ Duplicates removed!")

# Validate the optimized data
print("\n🔍 Validating optimized data...")
validation_issues = validate_data(optimized_data)

if validation_issues:
    print(f"⚠️  Found {len(validation_issues)} validation issues:")
    for issue in validation_issues:
        print(f"  - {issue}")
else:
    print("✅ Data validation passed!")

# Add optimization notes
if "notes" in optimized_data:
    optimization_note = {
        "date": datetime.now().isoformat(),
        "action": "File optimized",
        "details": "Removed redundancies, organized structure, validated data"
    }
    if "general" not in optimized_data["notes"]:
        optimized_data["notes"]["general"] = []
    optimized_data["notes"]["general"].append(optimization_note)

print("\n✅ Final optimization complete!")

## Step 8: Save Optimized File

Save the optimized data and create a backup of the original.

In [ ]:
def save_optimized_file(data, output_filename):
    """
    Save the optimized data to a file.
    """
    try:
        # Create backup of original if it exists
        if os.path.exists(FILE_NAME):
            backup_name = f"{BACKUP_DIR}/LHM23_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            import shutil
            shutil.copy2(FILE_NAME, backup_name)
            print(f"✅ Backup created: {backup_name}")
        
        # Save optimized file
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Optimized file saved: {output_filename}")
        
        # Calculate file size
        file_size = os.path.getsize(output_filename)
        print(f"📊 File size: {file_size:,} bytes ({file_size/1024:.2f} KB)")
        
        return True
    except Exception as e:
        print(f"❌ Error saving file: {e}")
        return False

# Save the optimized file
success = save_optimized_file(optimized_data, OUTPUT_FILE)

if success:
    print("\n" + "="*50)
    print("🎉 OPTIMIZATION COMPLETE!")
    print("="*50)
    print(f"Your optimized file is ready: {OUTPUT_FILE}")
    print("\nNext steps:")
    print("1. Review the optimized file")
    print("2. Fill in any missing information")
    print("3. Use this notebook to maintain and update your data")
    print("4. Run this notebook regularly to keep your data clean")

## Step 9: Preview Optimized Structure

View a summary of the optimized file.

In [ ]:
def preview_structure(data, max_depth=2, current_depth=0, prefix=""):
    """
    Display a tree view of the data structure.
    """
    if current_depth >= max_depth:
        return
    
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{prefix}├── {key}")
            if isinstance(value, (dict, list)):
                count = len(value)
                type_name = "dict" if isinstance(value, dict) else "list"
                print(f"{prefix}│   └── ({type_name} with {count} items)")
                if current_depth < max_depth - 1:
                    preview_structure(value, max_depth, current_depth + 1, prefix + "│   ")
            else:
                value_preview = str(value)[:50]
                if len(str(value)) > 50:
                    value_preview += "..."
                print(f"{prefix}│   └── {value_preview}")
    elif isinstance(data, list) and data:
        print(f"{prefix}└── [List with {len(data)} items]")
        if current_depth < max_depth - 1 and data:
            preview_structure(data[0], max_depth, current_depth + 1, prefix + "    ")

print("📋 OPTIMIZED FILE STRUCTURE PREVIEW")
print("="*50)
preview_structure(optimized_data, max_depth=3)

print("\n" + "="*50)
print(f"Total sections: {len(optimized_data)}")
print(f"Last updated: {optimized_data.get('metadata', {}).get('last_updated', 'N/A')}")
print(f"Version: {optimized_data.get('metadata', {}).get('version', 'N/A')}")

## Step 10: Quick Update Functions

Convenience functions for common updates.

In [ ]:
# Quick update functions for easy maintenance

def quick_add_goal(goal_text, term="short_term"):
    """
    Quickly add a business goal.
    """
    global optimized_data
    if term in optimized_data["goals_metrics"]:
        optimized_data["goals_metrics"][term].append({
            "goal": goal_text,
            "added": datetime.now().isoformat(),
            "status": "active"
        })
        optimized_data["metadata"]["last_updated"] = datetime.now().isoformat()
        print(f"✅ Goal added to {term}!")
        return True
    return False

def quick_add_note(note_text, category="general"):
    """
    Quickly add a note.
    """
    global optimized_data
    if category in optimized_data["notes"]:
        optimized_data["notes"][category].append({
            "note": note_text,
            "timestamp": datetime.now().isoformat()
        })
        optimized_data["metadata"]["last_updated"] = datetime.now().isoformat()
        print(f"✅ Note added to {category}!")
        return True
    return False

def quick_add_action_item(action_text, priority="medium"):
    """
    Quickly add an action item.
    """
    global optimized_data
    optimized_data["notes"]["action_items"].append({
        "action": action_text,
        "priority": priority,
        "created": datetime.now().isoformat(),
        "status": "pending"
    })
    optimized_data["metadata"]["last_updated"] = datetime.now().isoformat()
    print(f"✅ Action item added with {priority} priority!")
    return True

def save_updates():
    """
    Save any updates made to the optimized data.
    """
    return save_optimized_file(optimized_data, OUTPUT_FILE)

print("✅ Quick update functions ready!")
print("\nExample usage:")
print("  quick_add_goal('Increase revenue by 20%', 'short_term')")
print("  quick_add_note('Met with potential client', 'general')")
print("  quick_add_action_item('Follow up with Bob', 'high')")
print("  save_updates()  # Don't forget to save!")

## Usage Examples

Here are some examples of how to use this notebook to maintain your LHM & Bob source of truth.

In [ ]:
# Example 1: Add a business goal
print("Example 1: Adding a business goal")
print("-" * 40)
# quick_add_goal("Launch new product line by Q3", "short_term")
# save_updates()
print("Uncomment the lines above to add a goal\n")

# Example 2: Add a note
print("Example 2: Adding a note")
print("-" * 40)
# quick_add_note("Discussed partnership opportunities with XYZ Corp", "general")
# save_updates()
print("Uncomment the lines above to add a note\n")

# Example 3: Add an action item
print("Example 3: Adding an action item")
print("-" * 40)
# quick_add_action_item("Review Q1 financials", "high")
# save_updates()
print("Uncomment the lines above to add an action item\n")

# Example 4: Add custom data
print("Example 4: Adding custom data")
print("-" * 40)
print("# optimized_data['business_info']['mission'] = 'Your mission statement'")
print("# save_updates()")
print("Uncomment and modify to add custom data\n")

print("💡 TIP: Uncomment and run these examples to start populating your data!")

## Maintenance Checklist

**Regular Maintenance Tasks:**

### Weekly:
- [ ] Add new notes and action items
- [ ] Update action item statuses
- [ ] Review and update goals progress

### Monthly:
- [ ] Run full analysis to check for new redundancies
- [ ] Review and update business information
- [ ] Archive completed action items
- [ ] Update financial data

### Quarterly:
- [ ] Comprehensive review of all sections
- [ ] Update goals and KPIs
- [ ] Review and refine processes
- [ ] Clean up old notes

### Best Practices:
1. **Always save after making changes** - Run `save_updates()`
2. **Use descriptive text** - Make notes clear and searchable
3. **Date everything** - The system does this automatically
4. **Review backups** - Check the `backups/` folder periodically
5. **Keep it current** - Update regularly, even small changes
6. **Be consistent** - Use the same terminology throughout

---

**Your LHM23 file is now optimized and ready to grow with your business! 🚀**